In [6]:
import requests
from bs4 import BeautifulSoup
import json


# Send a request to the page and get the HTML content
def extract_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    doctor_details = []
    doctors_list = soup.find_all('div',class_='mydrbox')
    for doctor in doctors_list:
        # for name 
        name_tag = doctor.find('h5', class_='name')
        name = name_tag.text.strip() if name_tag else 'N/A'

        # for positionn and department 

        p_tag = doctor.find('p', class_='font-14 font-weight-500')

        position = p_tag.contents[0].strip()
        department = p_tag.contents[-1].strip()

        profile_link = soup.find('a', class_='btn')['href']

        doctor_url = 'https://www.lnh.edu.pk' + profile_link
        doctor_response = requests.get(doctor_url)
        
        doctor_soup = BeautifulSoup(doctor_response.content, 'html.parser')

        qualifications_tag = doctor_soup.find('h5', class_='text-theme-colored')
        qualifications = qualifications_tag.text.strip() if qualifications_tag else 'N/A'


        clinics = []
        panels = doctor_soup.find_all('div', class_='panel')
        for panel in panels:
            clinic_name_tag = panel.find('div', class_='panel-title')
            clinic_name = clinic_name_tag.text.strip() if clinic_name_tag else 'N/A'
            
            location_tag = panel.find('span', class_='font-weight-700')
            location = location_tag.find_next_sibling('span').text.strip() if location_tag else 'N/A'
            
            contact_tag = panel.find('span', text='Contact:')
            contact = contact_tag.find_next_sibling('span').text.strip() if contact_tag else 'N/A'
            
            timings = []
            rows = panel.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 3:
                    day = cols[0].text.strip()
                    shift = cols[1].text.strip()
                    timing = cols[2].text.strip()
                    timings.append({'day': day, 'shift': shift, 'timing': timing})
            
            clinics.append({
                'clinic_name': clinic_name,
                'location': location,
                'contact': contact,
                'timings': timings
            })
        
        doctor_details.append({
            'name': name,
            'qualifications': qualifications,
            'department': department,
            'hospital': 'Liaquat National Hospital',
            'clinics': clinics
        })

    return doctor_details   
doctor_details = []
# Save the data to a JSON file
with open("liaquat-national-hospital.txt", "r") as fs:
    urls = [line.strip() for line in fs.readlines()]




for url in urls:
    doctor_details.extend(extract_data(url))
print(doctor_details)

# # Save the data to a JSON file
with open('outputs/liaquat-national-hospital.json.json', 'w') as file:
    json.dump(doctor_details, file, indent=4)

# # Print the extracted data
for doctor in doctor_details:
    print(doctor)


/tmp/ipykernel_18720/3323552638.py:44: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  contact_tag = panel.find('span', text='Contact:')


[{'name': 'Dr. Hina Yousuf', 'qualifications': 'FCPS', 'department': 'Paediatric Surgery', 'hospital': 'Liaquat National Hospital', 'clinics': [{'clinic_name': 'Paediatrics Urology And Nephrology Clinic', 'location': 'Liaquat National Hospital', 'contact': '02134412474, 02134413036', 'timings': [{'day': 'Mon', 'shift': 'Morning', 'timing': '11:30 am  to  02:00 pm'}, {'day': 'Thu', 'shift': 'Morning', 'timing': '11:30 am  to  02:00 pm'}, {'day': 'Fri', 'shift': 'Afternoon', 'timing': '02:00 pm  to  04:30 pm'}, {'day': 'Sat', 'shift': 'Afternoon', 'timing': '02:00 pm  to  04:29 pm'}, {'day': 'Fri', 'shift': 'Morning', 'timing': '10:00 am  to  01:00 pm'}]}, {'clinic_name': 'Paediatrics Stone Management Clinic', 'location': 'Liaquat National Hospital', 'contact': '02134413199', 'timings': [{'day': 'Wed', 'shift': 'Evening', 'timing': '04:30 pm  to  06:00 pm'}, {'day': 'Sat', 'shift': 'Evening', 'timing': '04:30 pm  to  06:00 pm'}]}]}, {'name': 'Prof. Dr. Muhammad Arshad', 'qualifications':

In [9]:
with open('outputs/liaquat-national-hospital.json', 'w') as file:
    json.dump(doctor_details, file, indent=4)
